In [1]:
#default_exp neighbors

In [2]:
#hide
from nbdev.showdoc import *

#%load_ext autoreload
#%autoreload 2

import sys
sys.path.insert(0, '..') #appends project root to path in order to import project packages since `noteboks_dev` is not on the root


# Code

In [3]:
#export
from pathlib import Path
import time

import numpy as np
from scipy import sparse

import nmslib


from sklearn.base import BaseEstimator, TransformerMixin

from heartwood.utils import sparsify, hstack, vstack, stack, RobustEncoder

In [12]:
#export
class NMSLibSklearnWrapper(BaseEstimator):
        
    def __init__(
        self,
        #init index params
        nmslib_method='hnsw',
        nmslib_space='jaccard_sparse', 
        nmslib_data_type=nmslib.DataType.OBJECT_AS_STRING,
        nmslib_space_params = {},
        #index creation params
        index_time_params = {'M': 30, 'indexThreadQty': 4, 'efConstruction': 100, 'post' : 0},
        query_time_params = {'efSearch': 100},        
        #
        n_neighbors = 30,        
        verbose = False,
        #x_prep_function
        X_prep_function = None
    ):
        
                
        self.nmslib_method = nmslib_method
        self.nmslib_space=nmslib_space
        self.nmslib_data_type=nmslib_data_type
        self.nmslib_space_params = nmslib_space_params
        #index creation params
        self.index_time_params = index_time_params
        self.query_time_params = query_time_params
        #
        self.n_neighbors = n_neighbors
        self.verbose = verbose
        #x_prep_function
        self.X_prep_function = X_prep_function
        
        pass
    
    def _preprocess_X(self, X):
        '''
        encodes sparse rows into str of id of nonzero columns
        '''
        if not self.X_prep_function is None:
            X = self.X_prep_function(X)
        
        return X
        
    
    def _instantiate_index(self,):
        '''
        method for instantiating index.
        usefull for pickling
        '''         
        
        index = nmslib.init(
            method = self.nmslib_method,
            space = self.nmslib_space,
            data_type = self.nmslib_data_type,
            space_params = self.nmslib_space_params
        )
        
        return index
        
    def fit(self, X, y = None, **kwargs):
        '''
        instantiates and creates index
        '''
        #instantiate index
        index = self._instantiate_index()
        # preprocess X
        X_prep = self._preprocess_X(X)        
        #add points to index
        index.addDataPointBatch(X_prep)
        # Create an index        
        index.createIndex(self.index_time_params, self.verbose)
        #handle None for y (data to save under indexes)
        if y is None:
            y = np.ones((X.shape[0], 0)) #empty array
        # save states
        self.index_ = index        
        self.y_ = y
        self.X_ = X
        return self
    
    def partial_fit(self, X, y = None, **kwargs):
        '''
        adds new datapoints to index and y.
        estimator needs to be fit prior to calling partial fit,
        so first call fit in the first batch of data, then call partial fit 
        passing the subsequent batches
        '''        
        #assume index is already instantiated
        # preprocess X
        X_prep = self._preprocess_X(X)        
        #add points to index
        self.index_.addDataPointBatch(X_prep)
        # Create an index        
        self.index_.createIndex(self.index_time_params, self.verbose)
        #handle None for y (data to save under indexes)
        if y is None:
            y = np.ones((X.shape[0], 0)) #empty array
        # save states                
        self.y_ = vstack([self.y_, y])
        self.X_ = vstack([self.X_, X])
        return self
    
    def kneighbors(self, X = None, n_neighbors = None, return_distance = True, query_time_params = None, n_jobs = 4):
        '''
        query neighbors, if X is None, will return the neighbors of each point in index
        '''
        if query_time_params is None:
            query_time_params = self.query_time_params
        if n_neighbors is None:
            n_neighbors = self.n_neighbors
        if X is None:
            X = self.X_
        
        #preprocess X
        X = self._preprocess_X(X)            
        
        self.index_.setQueryTimeParams(query_time_params)

        # Querying                        
        start = time.time() 
        nbrs = self.index_.knnQueryBatch(X, k = n_neighbors, num_threads = n_jobs)
        end = time.time() 
        
        if self.verbose:
            try:
                query_qty = len(X)
            except:
                query_qty = X.shape[0]
            print('kNN time total=%f (sec), per query=%f (sec), per query adjusted for thread number=%f (sec)' % 
                  (end-start, float(end-start)/query_qty, n_jobs*float(end-start)/query_qty))
        
        if return_distance:
            distances = [nb[0] for nb in nbrs]
            nbrs = [nb[1] for nb in nbrs]
            return distances, nbrs
        else:
            nbrs = [nb[1] for nb in nbrs]
            return nbrs
                        
    def __getstate__(self,):
        '''
        creates binary file for index and then saves into object attribute to be pickled alongside
        other attributes.
        '''
        #read tempfiles with binaries to save binary str inside object
        tempfile_name = fr'.~nmslib_index_{str(int(time.time()*1e7))}'
        self.index_.saveIndex(tempfile_name, save_data = True)        
        
        with open(tempfile_name, 'rb') as f:
            fb = f.read()        
        with open(tempfile_name+'.dat', 'rb') as f:
            fb_dat = f.read()
        
        #save binary as attribute (index and data)
        self.index_ = (fb,fb_dat)
        #delete tempfiles
        Path(tempfile_name).unlink()
        Path(tempfile_name+'.dat').unlink()
        return self.__dict__
    
    def __setstate__(self,d):
        '''
        sets state during unpickling.
        instantiates index and loads binary index
        '''
        self.__dict__ = d
        
        #write tempfiles with binaries to load from index.loadIndex
        tempfile_name = fr'.~nmslib_index_{str(int(time.time()*1e7))}'        
        with open(tempfile_name, 'wb') as f:
            f.write(self.index_[0])
            
        with open(tempfile_name+'.dat', 'wb') as f:
            f.write(self.index_[1])
        
        index = self._instantiate_index()
        index.loadIndex(tempfile_name, load_data = True)
        #sets self.index_
        self.index_ = index
        #delete tempfile
        Path(tempfile_name).unlink()        
        Path(tempfile_name+'.dat').unlink()
        return

In [13]:
#export
def _preprocess_sparse_to_idx_str(X):
    '''
    encodes sparse rows into str of id of nonzero columns
    '''
    #ensure is sparse
    X = sparse.csr_matrix(X)
    indptr = X.indptr
    cols = X.tocoo().col.astype(str)
    id_strs = [*(' '.join(cols[slice(*indptr[i:i+2])]) for i in range(len(indptr)-1))]
    return id_strs


class FastJaccardNN(NMSLibSklearnWrapper):
        
    def __init__(
        self,
        n_neighbors = 30,
        index_time_params = {'M': 30, 'indexThreadQty': 4, 'efConstruction': 100, 'post' : 0},
        query_time_params = {'efSearch': 100},        
        verbose = False,
    ):
                        
        super().__init__(
            #jaccard init params
            nmslib_method='hnsw',
            nmslib_space='jaccard_sparse', 
            nmslib_data_type=nmslib.DataType.OBJECT_AS_STRING,
            nmslib_space_params = {},
            #other params
            X_prep_function = _preprocess_sparse_to_idx_str,
            n_neighbors = n_neighbors,
            index_time_params = index_time_params,
            query_time_params = query_time_params,
            verbose = verbose,                    
        )
        return
    
    

In [17]:
#testing
import joblib
X = joblib.load('terminal_node_embs')

In [15]:
jcd = FastJaccardNN(verbose = True).fit(tree_embeddings)
jcd.partial_fit(X)
results = jcd.kneighbors(n_jobs = 8)

kNN time total=3.225677 (sec), per query=0.000161 (sec), per query adjusted for thread number=0.001290 (sec)


In [19]:
joblib.dump(jcd,'jcd.sav')
jcd = joblib.load('jcd.sav')
#appends to index
jcd.partial_fit(X)
results = jcd.kneighbors(n_jobs = 8)

kNN time total=4.605462 (sec), per query=0.000154 (sec), per query adjusted for thread number=0.001228 (sec)


# Export

In [20]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted index.ipynb.
Converted kernel.ipynb.
Converted neighbors.ipynb.
Converted utils.ipynb.
